# LCLS cu_inj nn 

This example assumes the steps in the README have been completed, the model has been saved, and a deployment has been stored in the database. We will first load the model locally using the LocalModelingService and subsequently run with prefect using the RemoteModelingService.

The model accepts and outputs the following variables:

| Input variables               | Output variables               |
|-------------------------------|--------------------------------|
| distgen:r_dist:sigma_xy:value | end_n_particle                 |
| distgen:t_dist:length:value   | end_mean_gamma                 |
| distgen:total_charge:value    | end_sigma_gamma                |
| SOL1:solenoid_field_scale     | end_mean_x                     |
| CQ01:b1_gradient              | end_mean_y                     |
| SQ01:b1_gradient              | end_norm_emit_x                |
| L0A_phase:dtheta0_deg         | end_norm_emit_y                |
| L0A_scale:voltage             | end_norm_emit_z                |
| end_mean_z                    | end_sigma_x                    |
|                               | end_sigma_y                    |
|                               | end_sigma_z                    |
|                               | end_mean_px                    |
|                               | end_mean_py                    |
|                               | end_mean_pz                    |
|                               | end_sigma_px                   |
|                               | end_sigma_py                   |
|                               | end_sigma_pz                   |
|                               | end_higher_order_energy_spread |
|                               | end_cov_x__px                  |
|                               | end_cov_y__py                  |
|                               | end_cov_z__pz                  |
|                               | x:y                            |
|                               | out_ymax                       |
|                               | out_xmax                       |
|                               | out_ymin                       |
|                               | out_xmin                       |


## First, let's load the local modeling service from the service container

In [1]:
from slac_services import service_container
local_modeling_service = service_container.local_modeling_service()

## Use the model_id to retrieve latest model version
In a previous step, this was stored as an environment variable with the name $LCLS_CU_INJ_NN_MODEL_ID

In [2]:
import os

model_id = os.environ["LCLS_CU_INJ_NN_MODEL_ID"]
model = local_modeling_service.get_latest_model(model_id)
model

2022-03-18 14:59:54.841024: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Run model with dummy values

In [3]:
import copy
vars = copy.deepcopy(model.input_variables)
for var_name, var in vars.items():
    var.value = var.default // 2 

output = local_modeling_service.predict(model_id=model_id, input_variables=list(vars.values()))
output

[ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_n_particle', value=10059.875, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_mean_gamma', value=131.49306, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_sigma_gamma', value=2.5961263, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_mean_x', value=1.3658382e-07, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_mean_y', value=9.111318e-09, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_norm_emit_x', value=4.6450496e-06, precision=None, de

## We can also predict directly with model service:

In [4]:
local_modeling_service.predict(model_id = model_id, input_variables = list(vars.values()))

[ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_n_particle', value=10059.875, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_mean_gamma', value=131.49306, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_sigma_gamma', value=2.5961263, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_mean_x', value=1.3658382e-07, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_mean_y', value=9.111318e-09, precision=None, default=None),
 ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_norm_emit_x', value=4.6450496e-06, precision=None, de

#  Execute flow

So far, we've used the model object directly, however the distribution is packaged with a flow that provides the ability to format EPICS pvs before running the execution.
format_epics:

format_epics_input -> model_predict

This accepts three dictionary Prefect flow parameters:
- pv_values
- pvname_to_input_map
- settings

PV values are mapped to input variables using the map. Alternatively, the values for input variables may be set directly using settings.

In [5]:
pvname_to_input_map = {
    "IRIS:LR20:130:CONFG_SEL": "distgen:r_dist:sigma_xy:value",
    "BPMS:IN20:221:TMIT": "distgen:total_charge:value",
    "SOLN:IN20:121:BACT": "SOL1:solenoid_field_scale",
    "QUAD:IN20:121:BACT": "CQ01:b1_gradient",
    "QUAD:IN20:122:BACT": "SQ01:b1_gradient",
    "ACCL:IN20:300:L0A_PDES": "L0A_phase:dtheta0_deg",
    "ACCL:IN20:400:L0B_PDES": "L0A_scale:voltage"

}

pv_values = {
    "IRIS:LR20:130:CONFG_SEL" :  0.4130,
    "BPMS:IN20:221:TMIT": 250.0,
    "SOLN:IN20:121:BACT" : 0.2460,
    "QUAD:IN20:121:BACT": -0.0074,
    "QUAD:IN20:122:BACT": -0.0074,
    "ACCL:IN20:300:L0A_PDES":  -8.8997,
    "ACCL:IN20:400:L0B_PDES": 70000000.0
}

settings = {
    "distgen:t_dist:length:value" : 7.499772441611215,
    "end_mean_z": 4.6147002
}

data = {
    "pvname_to_input_map": pvname_to_input_map,
    "pv_values": pv_values,
    "settings": settings
}

local_modeling_service.predict_flow(model_id = model_id, data=data)

[2022-03-18 15:00:36-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'lcls-cu-inj-nn-ex'
[2022-03-18 15:00:36-0700] INFO - prefect.TaskRunner | Task 'pvname_to_input_map': Starting task run...
[2022-03-18 15:00:37-0700] INFO - prefect.TaskRunner | Task 'pvname_to_input_map': Finished task run for task with final state: 'Success'
[2022-03-18 15:00:37-0700] INFO - prefect.TaskRunner | Task 'pv_values': Starting task run...
[2022-03-18 15:00:37-0700] INFO - prefect.TaskRunner | Task 'pv_values': Finished task run for task with final state: 'Success'
[2022-03-18 15:00:37-0700] INFO - prefect.TaskRunner | Task 'settings': Starting task run...
[2022-03-18 15:00:37-0700] INFO - prefect.TaskRunner | Task 'settings': Finished task run for task with final state: 'Success'
[2022-03-18 15:00:37-0700] INFO - prefect.TaskRunner | Task 'format_epics_input': Starting task run...
[2022-03-18 15:00:37-0700] INFO - prefect.TaskRunner | Task 'format_epics_input': Finished task run for task with f

<Success: "All reference tasks succeeded.">

# Now, lets use the remote modeling service.
This specific model accepts three different parameters for running a remote flow, each optional but all input variables must be covered by the union of the three:

1. pv_values: dict of process variable name to values for use in composing input variables
2. pvname_to_input_map: 1-to-1 map of pv name to input name
3. settings: map of input variable name to value

In [9]:
remote_modeling_service = service_container.remote_modeling_service()
remote_modeling_service.predict(model_id=model_id, data=data)

# Open Prefect UI
You can now check the status of your run at:
http://localhost:8080